In [ ]:
display_width = 100 # in percentage

from IPython.display import display, HTML
display(HTML("<style>.container { width:"+str(display_width)+"% !important; }</style>"))

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
# import sys
# sys.path.append("/Users/sudhanshugupta/Library/Python/3.9/lib/python/site-packages")

In [ ]:
import pandas as pd
import numpy as np

import os

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from EDA import EDA_medical_appointment
from preprocessing import preprocMedicalAppointment
from feature_extraction import featuresMedicalAppointment, feature_selection_permutation_importance, feature_selection_hierarchical_clustering
from classification import LogisticRegressionClf, RandomForestClf, MLPClf, XGBClf
from utils import train_val_test_split

# Read Data

In [ ]:
df_medical_appointment = pd.read_csv("./data/medical_appointment_train.csv")
df_medical_appointment.rename(columns={"No-show": "no_show"}, inplace=True)
df_medical_appointment['no_show'].replace({"No": 0, "Yes": 1}, inplace=True)
feature_cols = [
    'PatientID', 'AppointmentID', 'Gender', 'ScheduledDay',
    'AppointmentDay', 'Age', 'Neighbourhood', 'Scholarship', 'Hypertension',
    'Diabetes', 'Alcoholism', 'Handicap', 'SMS_received',
]
df_medical_appointment.sample(10)

# Exploratory Data Analysis

In [ ]:
eda = EDA_medical_appointment(df_medical_appointment[feature_cols], df_medical_appointment['no_show'])

In [ ]:
eda.describe_general_stats()

Min age < 0 ==> incorrect entry

remove those values later

In [ ]:
eda.stats_per_unique_patient()

**Conclusion:** Frame the problem as a binary classification per patient

In [ ]:
eda.visulaize_class_distribution()

In [ ]:
eda.visulaize_no_show_prob_per_variable()

# Preprocessing

In [ ]:
preprocessor = preprocMedicalAppointment()

In [ ]:
print("before", df_medical_appointment.shape)
df_medical_appointment = preprocessor.remove_outliers_age(df_medical_appointment)
print("after", df_medical_appointment.shape)

In [ ]:
X_train, y_train, X_test, y_test, X_val, y_val = train_val_test_split(df_medical_appointment, feature_cols=feature_cols, target_col='no_show', test_percent=20)

# Feature Extraction

In [ ]:
feature_extractor = featuresMedicalAppointment()

In [ ]:
scheduled_after_appointment_strategy = 0
X_train = feature_extractor.feat_n_hours_scheduled_before(X_train, scheduled_after_appointment_strategy=scheduled_after_appointment_strategy)
X_train.shape

In [ ]:
# X_train = feature_extractor.feat_appointment_date(X_train)
# X_train.shape

In [ ]:
infrequent_threshold = 100
X_train = feature_extractor.feat_categorical_to_one_hot_encoding_train(X_train, infrequent_threshold=infrequent_threshold)
X_train.shape

In [ ]:
columns_to_scale = ['Age']
X_train = feature_extractor.feat_minmax_norm_train(X_train, columns_to_scale)
X_train.shape

In [ ]:
X_train.drop(columns=['PatientID', 'AppointmentID', 'ScheduledDay', 'AppointmentDay'], inplace=True)
X_train.shape

In [ ]:
X_train.info()

# Feature Selection

>**Permutation Importance**

    One of the most basic questions we might ask of a model is: 
    What features have the biggest impact on predictions? This concept is called feature importance.

    In this notebook we use permutation importance. Compared to most other approaches, permutation importance is:
           - fast to calculate,
           - widely used and understood, and
           - consistent with properties we would want a feature importance measure to have.
    Permutation importance is calculated after a model has been fitted.
    The basic question it answers is: If I randomly shuffle a single column of the validation data, leaving the target and all other columns in place, how would that affect the accuracy of predictions in that now-shuffled data? Randomly re-ordering a single column should cause less accurate predictions, since the resulting data no longer corresponds to anything observed in the real world. Model accuracy especially suffers if we shuffle a column that the model relied on heavily for predictions.


>**Hierarchical clustering**

    In hierarchical clustering, we look at every pair of objects and say which two objects are the closest. We then take the closest pair, delete them, and replace them with the midpoint of the two. Then repeat that again and again. Since we are removing points and replacing them with their averages, you are gradually reducing a number of points by pairwise combining. The cool thing is, you can plot that. Rather than looking at points, you look at variables and we can see which two variables are the most similar. So the horizontal axis here is how similar are the two points that are being compared. If they are closer to the right, that means that they are very similar. In this case, we actually used Spearman’s R. So correlation is almost exactly the same as the R², but it’s between two variables rather than a variable and its prediction. 

## Permutation Importance 1

In [ ]:
df_feature_importances = feature_selection_permutation_importance(
    X_train[:1000], 
    y_train[:1000], 
    n_jobs=4, 
    n_repeats=4,
    plot=True,
)

## Hierarchical Clustering 1

In [ ]:
THRESHOLD_CLUSTERING = 0.055
THRESHOLD_IMPORTANCE = None # Set to None in first iteration

In [ ]:
df_selected_features = feature_selection_hierarchical_clustering(
    X_train, 
    threshold_clustering=THRESHOLD_CLUSTERING, 
    threshold_importance=THRESHOLD_IMPORTANCE, 
    df_feature_importances=df_feature_importances, 
    plot=True,
)

display(df_selected_features)

In [ ]:
FEATURES_PERMUTATION_IMPORTANCE_2 = df_selected_features['feature'].to_list()

## Permutation Importance 2

In [ ]:
df_feature_importances = feature_selection_permutation_importance(
    X_train[:1000],
    y_train[:1000],
    n_jobs=4,
    n_repeats=4,
    plot=True,
)

## Hierarchical Clustering 2

In [ ]:
THRESHOLD_CLUSTERING = 0.055
THRESHOLD_IMPORTANCE = 0.005 # Keeping 0.005 as the reference, change until you have desired number of features

In [ ]:
df_selected_features = feature_selection_hierarchical_clustering(
    X_train[FEATURES_PERMUTATION_IMPORTANCE_2], 
    threshold_clustering=THRESHOLD_CLUSTERING, 
    threshold_importance=THRESHOLD_IMPORTANCE, 
    df_feature_importances=df_feature_importances, 
    plot=True,
)

display(df_selected_features)

In [ ]:
FEATURES_LIST_SELECTED = df_selected_features['feature'].to_list()

# Classification

In [ ]:
X_test = feature_extractor.feat_n_hours_scheduled_before(X_test, scheduled_after_appointment_strategy=scheduled_after_appointment_strategy)
X_test = feature_extractor.feat_categorical_to_one_hot_encoding_test(X_test, infrequent_threshold=infrequent_threshold)
X_test = feature_extractor.feat_minmax_norm_test(X_test, columns_to_scale)
X_test.drop(columns=['PatientID', 'AppointmentID', 'ScheduledDay', 'AppointmentDay'], inplace=True)
X_test.shape

## Logistic Regression Classifier

In [ ]:
clf_logistic_regression = LogisticRegressionClf()
clf_logistic_regression.fit_classifier(X_train[FEATURES_LIST_SELECTED], y_train)

In [ ]:
y_pred = clf_logistic_regression.predict_classifier(X_test[FEATURES_LIST_SELECTED])
clf_logistic_regression.evaluate_classifier(y_test, y_pred)

## Random Forest Classifier

In [ ]:
clf_rf = RandomForestClf()
clf_rf.fit_classifier(X_train[FEATURES_LIST_SELECTED], y_train)

In [ ]:
y_pred = clf_rf.predict_classifier(X_test[FEATURES_LIST_SELECTED])
clf_rf.evaluate_classifier(y_test, y_pred)

### Hyperparameter Tuning

In [ ]:
param_space = {
    'n_estimators': (10, 500),
    'max_depth': (50, 200),
    'min_samples_split': (2, 100),
    'min_samples_leaf': (5, 100),
    'max_features': (0.2, 1.0),
}

clf_rf.hyperparmater_tuning(X_train[FEATURES_LIST_SELECTED], y_train, param_space, n_iter=50, cv=5, n_jobs=32)

In [ ]:
clf_rf.classifier

In [ ]:
y_pred = clf_rf.predict_classifier(X_test[FEATURES_LIST_SELECTED])
clf_rf.evaluate_classifier(y_test, y_pred)

## XGBoost Classifier

In [ ]:
clf_xgb = XGBClf()
clf_xgb.fit_classifier(X_train[FEATURES_LIST_SELECTED], y_train)

In [ ]:
y_pred = clf_xgb.predict_classifier(X_test[FEATURES_LIST_SELECTED])
clf_xgb.evaluate_classifier(y_test, y_pred)

In [ ]:
threshold_values = np.arange(0, 1, 0.05)
clf_xgb.predict_per_threshold(X_test[FEATURES_LIST_SELECTED], y_test, threshold_values)

## MLP Classifier

In [ ]:
clf_mlp = MLPClf()
clf_mlp.fit_classifier(X_train[FEATURES_LIST_SELECTED], y_train)

In [ ]:
y_pred = clf_mlp.predict_classifier(X_test[FEATURES_LIST_SELECTED])
clf_mlp.evaluate_classifier(y_test, y_pred)

In [ ]:
clf_mlp.predict_proba_classifier(X_test[FEATURES_LIST_SELECTED])

In [ ]:
y_pred = clf_mlp.predict_at_threshold(X_test[FEATURES_LIST_SELECTED], 0.4)
clf_mlp.evaluate_classifier(y_test, y_pred)

In [ ]:
threshold_values = np.arange(0, 1, 0.05)
clf_mlp.predict_per_threshold(X_test[FEATURES_LIST_SELECTED], y_test, threshold_values)

# Error analysis

In [ ]:
import giskard

In [ ]:
classifier = clf_rf

In [ ]:
giskard_model = giskard.Model(
    model=classifier.predict_proba_classifier,  # A prediction function that encapsulates all the data pre-processing steps and that could be executed with the dataset used by the scan.
    model_type="classification",  # Either regression, classification or text_generation.
    name="Medical Appointment",  # Optional
    classification_labels=np.unique(y_test),  # Their order MUST be identical to the prediction_function's output order
    feature_names=FEATURES_LIST_SELECTED,  # Default: all columns of your dataset
    # classification_threshold=0.5,  # Default: 0.5
)

giskard_dataset = giskard.Dataset(pd.concat([X_test, pd.Series(y_test, name='target')], axis=1))

# Then apply the scan
results = giskard.scan(giskard_model, giskard_dataset)


In [ ]:
display(results)

In [1]:
from datetime import datetime
datetime.now().strftime('%Y%m%d%H%M%S')

'20230828231643'